In [1]:
# import dependencies.
import pandas as pd
import gmaps
import requests
import numpy as np
from citipy import citipy 
# Import the API key.
from config import g_key
from config import weather_api_key1

In [2]:
# 3. Create a set of 2000 random latitude and longitude combinations.
lats = np.random.uniform(low=-90.000, high=90.000, size=2000)
lngs = np.random.uniform(low=-180.000, high=180.000, size=2000)
lat_lngs = zip(lats, lngs)
lat_lngs

In [3]:
# Add the latitudes and longitudes to a list.
coordinates = list(lat_lngs)

In [4]:
# Create a list for holding the cities.
cities = []
# 4. Identify nearest city for each latitude and longitude combination.
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name
    
    # If the city is unique, we will add it to the cities list.
    if city not in cities:
        cities.append(city)
# Print the city count to confirm sufficient count.
len(cities)

778

In [5]:
# 5. Starting URL for Weather Map API Call.
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key1


In [7]:
# Create an empty list to hold the weather data.
city_data = []
# Print the beginning of the logging.
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Create counters.
record_count = 1
set_count = 1

Beginning Data Retrieval     
-----------------------------


In [11]:
# 6. Loop through all the cities in the list; introducing "enumerate()"
# to reference both index "i" and the "city" in 1 for-loop - avoid 2 for-loops
for i, city in enumerate(cities):

    # Group cities in sets of 50 for logging purposes.
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
    # Create endpoint URL with each city.
    # removing the blank spaces in the city name and concatenating the city name 
    city_url = url + "&q=" + city.replace(" ","+")

    # Log the URL, record, and set numbers and the city.
    print(f"Processing Record {record_count} of Set {set_count} | {city}")
    # Add 1 to the record count.
    record_count += 1
# Run an API request for each of the cities.
    try:
        # Parse the JSON and retrieve data.
        city_weather = requests.get(city_url).json()
        # Parse out the needed data.
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]
        city_description = city_weather["weather"][0]["description"]
        # Convert the date to ISO standard.
        #city_date = datetime.utcfromtimestamp(city_weather["dt"]).strftime('%Y-%m-%d %H:%M:%S')
        # Append the city information into city_data list.
        city_data.append({"City": city.title(),
                          "Country": city_country,
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Current Description": city_description})
                          #"Date": city_date})

# If an error is experienced, skip the city.
    except:
        print("City not found. Skipping...")
        pass

# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

Set 22 | marv dasht
City not found. Skipping...
Processing Record 29 of Set 22 | sorong
Processing Record 30 of Set 22 | tyukhtet
Processing Record 31 of Set 22 | vestmanna
Processing Record 32 of Set 22 | miri
Processing Record 33 of Set 22 | huarmey
Processing Record 34 of Set 22 | hirado
Processing Record 35 of Set 22 | iqaluit
Processing Record 36 of Set 22 | mountain home
Processing Record 37 of Set 22 | kirakira
Processing Record 38 of Set 22 | sturgis
Processing Record 39 of Set 22 | kinna
Processing Record 40 of Set 22 | dengzhou
Processing Record 41 of Set 22 | makokou
Processing Record 42 of Set 22 | warri
Processing Record 43 of Set 22 | korla
Processing Record 44 of Set 22 | lengshuijiang
Processing Record 45 of Set 22 | farah
Processing Record 46 of Set 22 | south daytona
Processing Record 47 of Set 22 | phalodi
Processing Record 48 of Set 22 | hachinohe
Processing Record 49 of Set 22 | walvis bay
Processing Record 50 of Set 22 | arrifes
Processing Record 1 of Set 23 | bal

In [12]:
# 7. Convert the array of dictionaries to a Pandas DataFrame.
city_data_df = pd.DataFrame(city_data)
city_data_df.head()

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,New Norfolk,AU,-42.7826,147.0587,54.90,50,86,4.00,overcast clouds
1,Tebingtinggi,ID,3.3285,99.1625,87.85,52,91,4.63,overcast clouds
2,Kodiak,US,57.7900,-152.4072,62.11,39,1,10.36,clear sky
3,Carupano,VE,10.6678,-63.2585,75.60,90,3,8.28,clear sky
4,Hermanus,ZA,-34.4187,19.2345,57.90,63,87,6.24,overcast clouds


In [13]:
len(city_data)

717

In [15]:
# Create the output file (CSV).
output_data_file = "WeatherPy_Database.csv"
# 8. Export the City_Data into a CSV.
city_data_df.to_csv(output_data_file, index_label="City_ID")